# Análise de reclamações
Buscamos compreender quais são as empresas mais odiadas.

Autor: Gabriel Bhering

#### Bibliotecas Utilizadas

In [1]:
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import path

Os dados são provenientes do [kaggle](https://www.kaggle.com/gerosa/procon), e um [script](https://gist.github.com/bhering/d21097f5b125d65fd8c422d81695d7f7) foi rodado para higieniza-los. O que o script faz é remover algumas incoerencias (como traços no campo de CEP), e apagar campos sem dados que estavam preenchidos com `NULL` ou `Nao se aplica`. 

Todos os .csv da pasta são lidos e compilados em um único dataframe `df`...

In [2]:
all_files = glob(path.join('./datasets/costumer_complaints',"*.csv"))

In [3]:
df = pd.concat((pd.read_csv(f) for f in all_files))
df['CNPJ'] = df['NumeroCNPJ'].map(str) + df['RadicalCNPJ'].map(str)

In [4]:
df[['AnoCalendario', 'DataAbertura', 'strNomeFantasia', 'Atendida', 'DescricaoAssunto', 'DescricaoProblema']][:10]

,AnoCalendario,DataAbertura,strNomeFantasia,Atendida,DescricaoAssunto,DescricaoProblema
0,2012,2009-09-17 10:02:24.000,SAMSUNG,N,"Telefone ( Convencional, Celular, Interfone, E...","Garantia (Abrangência, cobertura, etc.)"
1,2012,2009-09-21 09:01:49.000,AMAZON PC,S,Microcomputador / Produtos de Informática,"Garantia (Abrangência, cobertura, etc.)"
2,2012,2009-09-22 09:49:56.000,AON AFFINITY DO BRASIL,N,Seguro,Não pagamento de indenização (seguros em geral)
3,2012,2009-10-23 15:58:49.000,BOM PRECO,N,Microcomputador / Produtos de Informática,"Garantia (Abrangência, cobertura, etc.)"
4,2012,2009-05-15 12:33:24.000,SUDAMERIS,N,Financeira,Cobrança indevida.
5,2012,2010-02-11 16:35:27.000,NaN,S,Empresa de Cobrança,Cobrança indevida.
6,2012,2010-03-08 12:25:24.000,NaN,S,"Telefone ( Convencional, Celular, Interfone, E...",Falta de peca de reposição
7,2012,2005-08-11 10:16:23.000,SIEMENS CELULARES,N,"Telefone ( Convencional, Celular, Interfone, E...",Falta de peca de reposição
8,2012,2006-11-07 09:07:41.000,LG,S,Televisão / Vídeo Cassete / Filmadora / Video-...,"Garantia (Abrangência, cobertura, etc.)"
9,2012,2010-02-18 16:27:20.000,DAKO / GE,S,Fogão e Microondas,Produto com vício


In [5]:
print('Total de registros:', len(df))

Total de registros: 603036


In [6]:
print('Total de empresas:', df['CNPJ'].nunique())

Total de empresas: 58254


### Como descrever o quão detestada uma empresa é?
Podemos começar simplesmente observando a distribuição das empresas com mais reclamacões:

In [13]:
lim=20

In [7]:
print(df['strRazaoSocial'].value_counts(normalize=True)[:20])
print("Total:", df['strRazaoSocial'].value_counts(normalize=True)[:20].sum())

TELEMAR NORTE LESTE S/A                                  0.015167
SAMSUNG ELETRONICA DA AMAZONIA LTDA                      0.012129
TNL PCS S/A                                              0.008558
CLARO S.A.                                               0.008011
CAIXA ECONOMICA FEDERAL                                  0.007658
CLARO S/A                                                0.007597
ELECTROLUX DO BRASIL S/A                                 0.006975
SKY BRASIL SERVICOS LTDA                                 0.006640
TELEFONICA BRASIL S/A                                    0.006179
TIM CELULAR S/A                                          0.006093
VIA VAREJO S/A                                           0.006011
DIGIBRAS INDUSTRIA DO BRASIL S/A                         0.005967
MOTOROLA INDUSTRIAL LTDA                                 0.005955
TIM CELULAR S.A.                                         0.005686
MAGAZINE LUIZA S/A                                       0.005610
NOKIA DO B

Das 58254 empresas, as 20 empresas com mais reclamações tem 13,9% do total de 603036reclamações registradas.

Os membros da lista não são surpredentes: temos todas as provedoras de telefonia, internet e TV, bem como fabricantes de celular, alguns bancos, grandes distribuídoras de varejo. Um pouco mais surpreendente é o banco de empréstimos (BV) e a seguradora (Cardif) que aparecem entre os 20 primeiros lista.

Porém, embora muitas reclamações possam não ser boas para uma empresa, é ainda pior e mais fácil de classificar como ṕdio quando as empresas tem muitas reclamações

In [14]:
print(df[df.Atendida == 'N']['strRazaoSocial'].value_counts(normalize=True)[:lim])
print("Total:", df[df.Atendida == 'N']['strRazaoSocial'].value_counts(normalize=True)[:lim].sum())

TELEMAR NORTE LESTE S/A                                  0.008816
CAIXA ECONOMICA FEDERAL                                  0.008218
SAMSUNG ELETRONICA DA AMAZONIA LTDA                      0.007862
BV FINANCEIRA SA CREDITO FINANCIAMENTO E INVESTIMENTO    0.007136
RICARDO ELETRO DIVINOPOLIS LTDA                          0.006661
NOKIA DO BRASIL TECNOLOGIA LTDA                          0.005914
TNL PCS S/A                                              0.005597
CLARO S.A.                                               0.005492
TELEFONICA BRASIL S/A                                    0.005285
CLARO S/A                                                0.005210
BANCO SANTANDER (BRASIL) S.A.                            0.005061
VIA VAREJO S/A                                           0.004929
DIGIBRAS INDUSTRIA DO BRASIL S/A                         0.004788
BANCO ITAUCARD S/A                                       0.004788
SKY BRASIL SERVICOS LTDA                                 0.004784
CAIXA ECON

In [18]:
print(df[df.Atendida == 'N']['Atendida'].count())

227430


"TELEMAR NORTE LESTE S/A" (uma das subdivisões da Oi) no topo novamente! Indiscutivelmente, quantativamente a empresa mais detestada. Poderia encerrar o relatório por aqui, mas podemos ter mais insights especialmente se quebrarmos essas duas medidas por segmento e por tempo.